# Validação de Conectividade e Acesso a Dados (Diagnóstico)

Este notebook executa comandos SQL diretos para validar se a sessão Spark consegue acessar o catálogo e as tabelas.
Isso ajuda a isolar se o problema é código Python (ContextManager) ou infraestrutura Spark.

In [0]:
%load_ext autoreload
%autoreload 2

from pyspark.sql import SparkSession

# Tenta obter a sessão ativa (Databricks) ou criar uma nova se necessário (Local)
try:
    spark = SparkSession.getActiveSession()
    if not spark:
        print("Nenhuma sessão ativa encontrada. Criando nova...")
        spark = SparkSession.builder.appName("Diagnostico").getOrCreate()
    print(f"Sessão Spark obtida: {spark}")
except Exception as e:
    print(f"[CRITICO] Falha ao obter SparkSession: {e}")

In [0]:
print("\n--- 1. Listando Schemas no Catálogo 'olist_dataset' ---")
try:
    spark.sql("SHOW SCHEMAS IN olist_dataset").limit(100).show(truncate=False)
except Exception as e:
    print(f"[ERRO] Falha ao listar schemas: {e}")

In [0]:
print("\n--- 2. Listando Tabelas nos Schemas do Catálogo 'olist_dataset' ---")
try:
    df_cx = spark.sql("SHOW TABLES IN olist_dataset.olist_cx")
    df_finance = spark.sql("SHOW TABLES IN olist_dataset.olist_finance")
    df_logistics = spark.sql("SHOW TABLES IN olist_dataset.olist_logistics")
    df_sales = spark.sql("SHOW TABLES IN olist_dataset.olist_sales")
    df_all = df_cx.unionByName(df_finance).unionByName(df_logistics).unionByName(df_sales)
    display(df_all)
except Exception as e:
    print(f"[ERRO] Falha ao listar tabelas: {e}")

In [0]:
print("\n--- 3. Amostra de Dados de 'olist_dataset.olist_cx.order_reviews' ---")
try:
    spark.sql("SELECT * FROM olist_dataset.olist_cx.order_reviews").limit(5).show(truncate=False)
except Exception as e:
    print(f"[ERRO] Falha ao ler dados da tabela: {e}")

In [0]:
print("\n--- 4. Teste de Coleta para Driver (toPandas) ---")
# Esse teste verifica se há problemas de comunicação Driver <-> Executor (erros de socket)
try:
    df = spark.sql("SELECT * FROM olist_dataset.olist_cx.order_reviews LIMIT 5")
    pdf = df.toPandas()
    print("Sucesso ao converter para Pandas:")
    print(pdf.head())
except Exception as e:
    print(f"[ERRO] Falha ao coletar dados para o Driver (toPandas): {e}")